In [ ]:
! pip3 install xgboost
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [ ]:
 xgb = XGBClassifier( objective = "multi:softmax", use_label_encoder=False, seed=10,
                         gamma=0,
                         subsample=0.8,
                         colsample_bytree=0.8,
                         learning_rate =0.1, 
                         n_estimators=240, 
                        max_depth = 9,
                        min_child_weight = 6.0,
                        num_class=3, 
                        verbose=0
                         )

In [ ]:
    le = LabelEncoder()
    y = le.fit_transform(y=train_labels.values.ravel())

In [ ]:
xgb.fit(train_values, y)

In [ ]:
test = pd.read_csv("Archivos auxiliares KNN/test_values_short1.csv", index_col='building_id')

In [ ]:
preds = xgb.predict(test)
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')
submission_horrible = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=submission_format.index)

submission_mejorada = submission_horrible + 1
submission_mejorada.to_csv('xgboost_knn_sin_outliers.csv', index=True)